<a href="https://colab.research.google.com/github/alecbidaran/Pytorch_excersies/blob/main/pytorch_Duel_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ptan
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym[atari]
!pip install pyvirtualdisplay
!pip install piglet
!pip install pytorch-ignite


     |████████████████████████████████| 776.8MB 23kB/s 
  Created wheel for ptan: filename=ptan-0.7-cp37-none-any.whl size=23504 sha256=248f8e4fd7558894e70ef8680c70063f1c8d10f97a6510d7176c1e6663213a7d
  Stored in directory: /root/.cache/pip/wheels/2c/58/0c/a42dad12a5cc0e130453042707b3e2205adfb901ae35cfad75
Successfully built ptan
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.7.0 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.7.0 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB o

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ROMS/Assau

In [ ]:
!apt-get install xdpyinfo

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package xdpyinfo


In [ ]:
try:
    import pyvirtualdisplay
    display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
except ImportError:
    pass

In [ ]:
import gym 
import collections
import ptan 
import numpy as np 
import matplotlib.pyplot as plt
import torch 
from typing import Iterable, Tuple, List

In [ ]:
class Dueling_DQN(torch.nn.Module):
  def __init__(self,input_shape,n_action):
    super(Dueling_DQN,self).__init__()
    self.convnet=torch.nn.Sequential(
        torch.nn.Conv2d(input_shape[0],32,kernel_size=8,stride=4),
        torch.nn.ReLU(),
        torch.nn.Conv2d(32,64,kernel_size=4,stride=2),
        torch.nn.ReLU(),
        torch.nn.Conv2d(64,64,kernel_size=3,stride=1),
        torch.nn.ReLU(),
    )
    self.flatten=torch.nn.Flatten()
    self.convout=self._get_conv_out(input_shape)
    self.val=torch.nn.Sequential(
        torch.nn.Linear(self.convout,256),
        torch.nn.ReLU(),
        torch.nn.Linear(256,1),
    )
    self.adv=torch.nn.Sequential(
        torch.nn.Linear(self.convout,256),
        torch.nn.ReLU(),
        torch.nn.Linear(256,n_action),
    )
  def _get_conv_out(self, shape):
        o = self.convnet(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
  def get_value_adv(self,x):
    fx=x.float()/256
    y=self.convnet(fx)
    y=torch.nn.Flatten()(y)
    return self.val(y),self.adv(y)
  def forward(self,x):
    val,adv=self.get_value_adv(x)
    return val+(adv-adv.mean(dim=1,keepdim=True))


In [ ]:
def unpack_batch(batch):
    states, actions, rewards, dones, last_states = [],[],[],[],[]
    for exp in batch:
        state = np.array(exp.state)
        states.append(state)
        actions.append(exp.action)
        rewards.append(exp.reward)
        dones.append(exp.last_state is None)
        if exp.last_state is None:
            lstate = state  # the result will be masked anyway
        else:
            lstate = np.array(exp.last_state)
        last_states.append(lstate)
    return np.array(states, copy=False), np.array(actions), \
           np.array(rewards, dtype=np.float32), \
           np.array(dones, dtype=np.uint8), \
           np.array(last_states, copy=False)


def calc_loss_dqn(batch, net, tgt_net, gamma, device="cuda"):
    states, actions, rewards, dones, next_states = \
        unpack_batch(batch)

    states_v = torch.tensor(states).to(device)
    next_states_v = torch.tensor(next_states).to(device)
    actions_v = torch.tensor(actions).to(device)
    rewards_v = torch.tensor(rewards).to(device)
    done_mask = torch.BoolTensor(dones).to(device)

    actions_v = actions_v.unsqueeze(-1)
    state_action_vals = net(states_v).gather(1, actions_v)
    state_action_vals = state_action_vals.squeeze(-1)
    with torch.no_grad():
        next_state_vals = tgt_net(next_states_v).max(1)[0]
        next_state_vals[done_mask] = 0.0

    bellman_vals = next_state_vals.detach() * gamma + rewards_v
    return torch.nn.MSELoss()(state_action_vals, bellman_vals)
    
def batch_generator(buffer,initial, batch_size):
    buffer.populate(initial)
    while True:
        buffer.populate(1)
        yield buffer.sample(batch_size)


In [ ]:
class EpsilonTracker:
  def __init__(self,selector:ptan.actions.EpsilonGreedyActionSelector):
    self.selector=selector
    self.final_epsilon=0.02
    self.frame(0)
  def frame(self,frame_idx:int):
    eps=1.0-frame_idx/10**5
    self.selector.epsilon=max(self.final_epsilon,eps)

In [ ]:
BATCH_SIZE = 32
TGT_NET_SYNC = 1000
GAMMA = 0.99
REPLAY_SIZE = 100000
LR = 1e-4
EPS_DECAY=0.99
NAME = "06_dueling"
STATES_TO_EVALUATE = 1000
EVAL_EVERY_FRAME = 100
env=gym.make("PongNoFrameskip-v4")
env=ptan.common.wrappers.wrap_dqn(env)
observation_size=env.observation_space.shape
action_size=env.action_space.n


In [ ]:
net=Dueling_DQN(input_shape=observation_size,n_action=action_size).to("cuda")

In [ ]:
tgt_net=ptan.agent.TargetNet(net)
selector=ptan.actions.EpsilonGreedyActionSelector(epsilon=1.0)
agent=ptan.agent.DQNAgent(net,action_selector=selector,device='cuda')
exp_source=ptan.experience.ExperienceSourceFirstLast(env,agent,gamma=GAMMA,steps_count=3)
buffer=ptan.experience.ExperienceReplayBuffer(exp_source,buffer_size=REPLAY_SIZE)
optimizer=torch.optim.Adam(net.parameters(),lr=LR)
exp_epsilon=EpsilonTracker(selector=selector)


In [ ]:
import ptan.ignite as ptan_ignite
from ignite.engine import Engine
from ignite.metrics import RunningAverage
from ignite.contrib.handlers import tensorboard_logger as tb_logger
from types import SimpleNamespace
import warnings
from datetime import timedelta, datetime

In [ ]:
@torch.no_grad()
def evaluate_states(states, net, device, engine):
    s_v = torch.tensor(states).to(device)
    adv, val = net.get_value_adv(s_v)
    engine.state.metrics['adv'] = adv.mean().item()
    engine.state.metrics['val'] = val.mean().item()

In [ ]:
def setup_ignite(engine: Engine, params: SimpleNamespace,
                 exp_source, run_name: str,
                 extra_metrics: Iterable[str] = ()):
    # get rid of missing metrics warning
    warnings.simplefilter("ignore", category=UserWarning)

    handler = ptan_ignite.EndOfEpisodeHandler(
        exp_source, bound_avg_reward=params.stop_reward)
    handler.attach(engine)
    ptan_ignite.EpisodeFPSHandler().attach(engine)

    @engine.on(ptan_ignite.EpisodeEvents.EPISODE_COMPLETED)
    def episode_completed(trainer: Engine):
        passed = trainer.state.metrics.get('time_passed', 0)
        print("Episode %d: reward=%.0f, steps=%s, "
              "speed=%.1f f/s, elapsed=%s" % (
            trainer.state.episode, trainer.state.episode_reward,
            trainer.state.episode_steps,
            trainer.state.metrics.get('avg_fps', 0),
            timedelta(seconds=int(passed))))

    @engine.on(ptan_ignite.EpisodeEvents.BOUND_REWARD_REACHED)
    def game_solved(trainer: Engine):
        passed = trainer.state.metrics['time_passed']
        print("Game solved in %s, after %d episodes "
              "and %d iterations!" % (
            timedelta(seconds=int(passed)),
            trainer.state.episode, trainer.state.iteration))
        trainer.should_terminate = True

    now = datetime.now().isoformat(timespec='minutes').replace(':', '')
    logdir = f"runs/{now}-{params.run_name}-{run_name}"
    tb = tb_logger.TensorboardLogger(log_dir=logdir)
    run_avg = RunningAverage(output_transform=lambda v: v['loss'])
    run_avg.attach(engine, "avg_loss")

    metrics = ['reward', 'steps', 'avg_reward']
    handler = tb_logger.OutputHandler(
        tag="episodes", metric_names=metrics)
    event = ptan_ignite.EpisodeEvents.EPISODE_COMPLETED
    tb.attach(engine, log_handler=handler, event_name=event)

    # write to tensorboard every 100 iterations
    ptan_ignite.PeriodicEvents().attach(engine)
    metrics = ['avg_loss', 'avg_fps']
    metrics.extend(extra_metrics)
    handler = tb_logger.OutputHandler(
        tag="train", metric_names=metrics,
        output_transform=lambda a: a)
    event = ptan_ignite.PeriodEvents.ITERS_100_COMPLETED
    tb.attach(engine, log_handler=handler, event_name=event)

In [ ]:
def process_batch(engine, batch):
        optimizer.zero_grad()
        loss_v = calc_loss_dqn(
            batch, net, tgt_net.target_model,
            gamma=params.gamma, device="cuda")
        loss_v.backward()
        optimizer.step()
        exp_epsilon.frame(engine.state.iteration)
        if engine.state.iteration % params.target_net_sync == 0:
            tgt_net.sync()
        if engine.state.iteration % EVAL_EVERY_FRAME==0:
          eval_state=getattr(engine.state,"eval_state",None)
          if eval_state is None:
            eval_state=buffer.sample(STATES_TO_EVALUATE)
            eval_state=[np.array(translation.state, copy=False) for translation in eval_state]
            eval_state=np.array(eval_state,copy=False)
            engine.state.eval_state=eval_state
            evaluate_state(eval_state,net,engine,device="cuda")
        return {"loss":loss_v.item(),
                "epsilon":selector.epsilon}


In [ ]:
HYPERPARAMS = {
    'pong': SimpleNamespace(**{
        'env_name':         "PongNoFrameskip-v4",
        'stop_reward':      18.0,
        'run_name':         'pong',
        'replay_size':      100000,
        'replay_initial':   10000,
        'target_net_sync':  1000,
        'epsilon_frames':   10**5,
        'epsilon_start':    1.0,
        'epsilon_final':    0.02,
        'learning_rate':    0.0001,
        'gamma':            0.99,
        'batch_size':       32
    })}
params=HYPERPARAMS['pong']
engine = Engine(process_batch)
setup_ignite(engine, params, exp_source, f"{NAME}={3}")
engine.run(batch_generator(buffer, params.replay_initial, params.batch_size))

Episode 1: reward=-18, steps=1024, speed=0.0 f/s, elapsed=0:00:23
Episode 2: reward=-20, steps=911, speed=0.0 f/s, elapsed=0:00:23
Episode 3: reward=-20, steps=925, speed=0.0 f/s, elapsed=0:00:23
Episode 4: reward=-21, steps=822, speed=0.0 f/s, elapsed=0:00:23
Episode 5: reward=-20, steps=1004, speed=0.0 f/s, elapsed=0:00:23
Episode 6: reward=-20, steps=906, speed=0.0 f/s, elapsed=0:00:23
Episode 7: reward=-21, steps=779, speed=0.0 f/s, elapsed=0:00:23
Episode 8: reward=-19, steps=934, speed=0.0 f/s, elapsed=0:00:23
Episode 9: reward=-21, steps=821, speed=0.0 f/s, elapsed=0:00:23
Episode 10: reward=-20, steps=950, speed=0.0 f/s, elapsed=0:00:23
Episode 11: reward=-21, steps=838, speed=0.0 f/s, elapsed=0:00:23
Episode 12: reward=-21, steps=923, speed=99.3 f/s, elapsed=0:00:31
Episode 13: reward=-21, steps=844, speed=99.3 f/s, elapsed=0:00:40
Episode 14: reward=-20, steps=923, speed=99.3 f/s, elapsed=0:00:49
Episode 15: reward=-20, steps=951, speed=99.3 f/s, elapsed=0:00:59
Episode 16: r

Engine run is terminating due to exception: 


KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard 
%tensorboard loads  runs

ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2021-07-04 08:59:24.521487: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
usage: tensorboard [-h] [--helpfull] {serve,dev} ...
tensorboard: error: invalid choice: 'loads' (choose from 'serve', 'dev')